# Introduction

This worksheet covers the three supervised learning algorithms we looked at in week 14: k-nearest neighbours, linear regression, and the naive Bayes classifier. Similar to last week, you will do some work implementing your own versions of these algorithms, to ensure that you understand the details of them. You will also compare them with the implementations in scikit-learn to test your implementations.

# Preliminaries
Import key packages: NumPy, matplotlib, and any others that you prefer to work with. In general, when writing code, you will put all your import statements at the top. However, for these worksheets we will import as we go along.

In [ ]:
#TODO: import NumPy and matplotlib here
import numpy as np
import matplotlib.pyplot as plt

# Question 1: k-nearest neighbours classification
In this question we will use the k-nearest neighbours algorithm to make predictions on the breast cancer Wisconsin dataset. This is a classification problem where the aim is to classify instances as either being malignant or benign based on the following 10 features:
1. radius (mean of distances from center to points on the perimeter)
2. texture (standard deviation of gray-scale values)
3. perimeter
4. area
5. smoothness (local variation in radius lengths)
6. compactness (perimeter squared/ area −1)
7. concavity (severity of concave portions of the contour)
8. concave points (number of concave portions of the contour)
9. symmetry
10. fractal dimension (‘coastline approximation’ −1)

In this question you will (a) download the dataset from sklearn and store the data and targets in suitable variables, (b) separate your data into a training and test split, (c) write your own function to implement k-nearest neighbours, (d) check your implementation with that of sklearn. Wethen go on to (e) select the most appropriate value of $k$ using cross-validation.

## Part (a)
Import the package `datasets` from `sklearn` and then load the breast cancer dataset (function is `load_breast_cancer()`). Save the data into a variable `X` and the targets into a variable `Y`. 
Take a look at the data in `X`. How many datapoints are there? How many features does each datapoint have? (Hint: use `np.shape`).
Take a look at the targets. Is this suitable for a classification algorithm or a regression algorithm?

In [ ]:
#TODO: import suitable packages, load the dataset, and save data and targets into variables X and Y
from sklearn import datasets

breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
Y = breast_cancer.target

## Part (b)

Use the function `train_test_split` from `sklearn.model_selection` to split your data into a training set and a held-out test set. Use a test set that is 0.2 of the original dataset. Set the parameter `random_state` to 10 to help with replication.

In [ ]:
# TODO: import the package train_test_split from sklearn.model_selection.
# Split the dataset into Xtr, Xtest, Ytr, Ytest. Xtest and Ytest will form your held-out
# test set. You will later split Xtr and Ytr into training and validation sets.
from sklearn.model_selection import train_test_split
Xtr, Xtest, Ytr, Ytest = train_test_split(X, Y, test_size = 0.2, random_state = 10)

## Part (c) 
Recall from the lecture that the k-nearest neighbours algorithm runs as follows:

Training step: Simply store the dataset

Prediction step: Given a datapoint $\vec{x}$:
 - **Find** the k datapoints $(\vec{x}_i, y_i)$ where the distance from $\vec{x}$ to $\vec{x}_i$ is smallest
 - **Return** the majority class from the $y_i$
   
 
What, if anything, do you need to do for the training step?

Write function(s) to implement the k-nearest neighbours prediction step. You may wish to break the procedure down into two functions `predict_datapoint` that makes a prediction for one datapoint and `predict_data` that loops over the whole dataset.

To select the majority class from the nearest neighbours, you can use the function `scipy.stats.mode()`


In [ ]:
from scipy.stats import mode
##TODO##: 
# Write function(s) to implement the prediction step in k-nearest neighbours. 
# You can use the suggested structure below if desired.

# predict_datapoint takes 4 arguments. pt (type: numpy array) is the datapoint we are making a prediction about, 
# Xtrain and Ytrain (numpy arrays) are training data and targets, k (int) is the number of neighbours.
# Returns an integer which is the predicted class for pt
def predict_datapoint(pt, Xtrain, Ytrain, k):
    
    # For each datapoint in Xtrain, calculate the distance to pt and store
    dists = np.linalg.norm(Xtrain-pt, axis=1)

        
    # Sort the list of distances (hint: use np.argsort)
    indices = np.argsort(dists)
    
    # obtain the classes (in Ytrain) of the datapoints with the smallest distance to pt
    classes = Ytrain[indices[:k]]
    
    # return the mode of the classes
    return mode(classes).mode[0]

# predict_data takes 4 arguments: the test data Xtst (numpy array), the training data Xtrain (numpy array),
# the training targets Ytrain (numpy array), and the number of neighbours k (int, default = 3). 
# Returns: predictions (array of int) for each point in Xtst
def predict_data(Xtst, Xtrain, Ytrain, k=3):
    #Loop over the datapoints in Xtst and store the prediction for that datapoint
    preds = []
    for x in Xtst:
        preds.append(predict_datapoint(x, Xtrain, Ytrain, k))
        
    # Return the predictions
    return preds

# Predict values for the TRAINING data (we will not look at the test set yet)
my_Ypred = predict_data(Xtr, Xtr, Ytr)

## Part (d)
Now we can compare your implementation with the sklearn implementation (you should get the same results). Firstly import the classfifier `KNeighborsClassifier` from `sklearn.neighbors`. Instantiate the classifier with the same number of neighbours that you used previously. Fit the model and make a prediction on the test set.

In [ ]:
##TODO##
# Import KNeighborClassifier
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the classifier with 3 neighbors
knn = KNeighborsClassifier(n_neighbors=3)

#Fit the classifier on the training data
model=knn.fit(Xtr, Ytr)

#Make a prediction on the training data
Ypred=model.predict(Xtr)

Check whether your predictions are the same as the predictions from `KNeighborsClassifier`.

In [ ]:
##TODO## Write your answer here
np.all(my_Ypred==Ypred)

Use the built in metrics in sklearn to calculate the accuracy of your classifier on the TRAINING set.

In [ ]:
##TODO## Write your answer here
from sklearn.metrics import accuracy_score
accuracytr=accuracy_score(Ytr,my_Ypred)
print(accuracytr)

## Part(e) Using cross-validation for model selection
k-nearest neighbours has the parameter $k$, and we need to decide which is the best value of $k$ to use. Last week we talked about using cross-validation for model selection.

We will use cross-validation on our training set to select the best value of $k$, in a range from 1 to 30.

NB: use sklearn's version of k-NN rather than yours, since unless you have optimised yours it is probably too slow.

Since we are using cross-validation for model selection we will cross-validate on the training set only.

Procedure:
        
 1. Import `KFold` from `sklearn.model_selection`  
 1. Instantiate `KFold` with 5 splits. Set the parameter `random_state` to help you reproduce your results if needed.
 1. Set a variable `max_k` to 30  
 1. Inititalise two variables to store the training accuracies and validation accuracies (these need to store max_k\*5 accuracies)  
 1. Loop over the values of k:  
    1. Instantiate a k-nn classifier (Use the sklearn classifier) with the current value of k  
    1. Loop over the cross-validation splits:  
       1. fit the model on the current split of data  
       1. make predictions  
       1. calculate training and validation accuracy and store  
 6. Calculate the mean training and validation accuracies across splits for each $k$

Plot the mean training and validation accuracies. Which value of $k$ will you use? Why?
    

In [ ]:
##TODO##
# Import KFold from sklearn.model_selection
from sklearn.model_selection import KFold

# Instantiate KFold with 5 splits. 
# Set the parameter random_state to help you reproduce your results if needed.
cv = KFold(n_splits=5,random_state=10, shuffle=True)

# Set a variable max_k to 30 
max_k = 30

# Inititalise two variables to store the 
# training accuracies and validation accuracies 
# (these need to store max_k*5 accuracies) 
trainaccuracy = [[] for _ in range(max_k)]
valaccuracy = [[] for _ in range(max_k)]

# Loop over the values of k: 
for k in range(max_k):
    
    # Instantiate a k-nn classifier (Use the sklearn classifier) with the current value of k 
    knn = KNeighborsClassifier(n_neighbors=k+1)
    # Loop over the cross-validation splits: 
    for train_index, val_index in cv.split(Xtr):
        Xtrain, Xval, Ytrain, Yval = Xtr[train_index],\
            Xtr[val_index], Ytr[train_index], Ytr[val_index]
        # fit the model on the current split of data 
        model=knn.fit(Xtrain, Ytrain)
        # make predictions 
        Ypredtrain=model.predict(Xtrain)
        Ypredval=model.predict(Xval)
        # calculate training and validation accuracy and store 
        trainaccuracy[k].append(accuracy_score(Ytrain,Ypredtrain))
        valaccuracy[k].append(accuracy_score(Yval,Ypredval))

# Calculate the mean training and validation accuracies across splits for each k
mean_train_acc = np.mean(trainaccuracy, axis=1)
mean_val_acc = np.mean(valaccuracy, axis=1)

In [ ]:
##TODO#
# Plot the mean training and validation accuracies. Which value of 𝑘 will you use? Why?
x = range(1, max_k+1)
plt.plot(x, mean_train_acc, label='Training Accuracy')
plt.plot(x, mean_val_acc, label='Validation Accuracy')
plt.legend()
plt.xlabel('k')
plt.ylabel('Accuracy')

# The naive Bayes classifier

Recall from the lecture notes that the naive Bayes classifier works as follows. We are trying to approximate an unknown function $$f:\Omega \rightarrow \mathcal{O}$$
where $\Omega$ is our feature space and our output space $\mathcal{O} = \{c_1, c_2, ... c_K\}$ is a finite set of classes.

The naive Bayes classifier does this by building a model that assigns the class label $\hat{y} = c_k$ as follows:
$$
\hat{y} = \text{argmax}_k p(c_k)\prod_i p(x_i| c_k)
$$
i.e., the $k$ that maximises this quantity.

In practice, multiplying all the $p(x_i| c_k)$ together is going to give some very small values. Therefore, we can take the log to make it easier to compute:
\begin{align}
\hat{y} &= \text{argmax}_k p(c_k)\prod_i p(x_i| c_k)= \text{argmax}_k log(p(c_k)\prod_i p(x_i| c_k))\\
&=\text{argmax}_k log(p(c_k)) + \sum_i log(p(x_i|c_k))
\end{align}

If we choose that $p(x_i|c_k)$ is given by a normal distribution with mean $\mu_k$ and variance $\sigma_k^2$, then we obtain the following expression:

\begin{align}
\hat{y} &=\text{argmax}_k \log(p(c_k)) + \sum_i \log(p(x_i|c_k))\\
&= \text{argmax}_k \log(p(c_k)) + \sum_i \log\left(\frac{1}{\sigma_k\sqrt{2\pi}} exp\left(\frac{-(x-\mu_k)^2}{2\sigma_k}\right)\right)\\
&= \text{argmax}_k \log(p(c_k)) - \sum_i \log\left(\sigma_k\sqrt{2\pi}\right) - \sum_i\left(\frac{(x-\mu_k)^2}{2\sigma_k}\right) \quad \text{ log-likelihood}
\end{align}

Expressing the values in terms of these sums means that they do not get so small, and it is less likely that there will be errors at the machine precision level.


How do we implement this in practice? We assume that each probability $p(x_i| c_k)$ is given by some distribution, and then given a datapoint $\vec{x}$, we plug the value into the equation for the distribution.

In this question you will (a) implement your own version of the Gaussian naive Bayes classifier, (b) check your classifier against the implementation in sci-kit learn, (c) compare the accuracy of the naive Bayes classifier with the accuracy of the k-nearest neighbours classifier, and (d) run cross-validation to verify whether the kNN classfier or the Gaussian naive Bayes classifier performs better on this dataset.

## Part (a) Implementing Gaussian naive Bayes
For this question we will make the assumption that each feature is described by a normal (also called Gaussian) distribution. The procedure is as follows:
1. Divide the training data by class
2. Calculate mean and standard deviation per class and per feature
4. For each datapoint in the validation set, calculate the log-likelihood for each class and for each feature (Hint: use the function `scipy.stats.norm.logpdf`)
5. Combine these values together with the probability of the class according to the log-likelihood equation above
6. Choose the class with the highest value


In [ ]:
##TODO##
# Write your own implementation of naive Bayes applied to the breast cancer dataset.

# If you wish you can follow the structure below
from scipy.stats import norm

# Split the training data into training and validation sets with an 80:20 split. 
# Set the random state to help with reproducibility
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtr, Ytr, test_size = 0.2, random_state=10)

#Separate the training set into classes, so you have one set of data for each class
data0 = Xtrain[Ytrain==0]
data1 = Xtrain[Ytrain==1]

# Calculate the means and standard deviations for each class, for each feature. 
# There are 30 features in the dataset, so you should have a 30-dimensional 
# array of means for each class and a 30-dimensional array of standard deviations
# for each class. Remember that you can take the average across rows or columns of 
# a matrix by specifying axis = 1 or axis = 0

means0 = np.mean(data0, axis=0)
stds0 = np.std(data0, axis=0)
means1 = np.mean(data1, axis=0)
stds1 = np.std(data1, axis=0)

# Calculate the prior probability p(c_i) for each class
prob_c0 = len(data0)/len(Xtrain)
prob_c1 = len(data1)/len(Xtrain)

# Calculate the log-likelihood of each class for each datapoint in the validation set
# Hint: you can use the function scipy.stats.norm.logpdf to help with this
ll_0 = np.log(prob_c0) + np.sum(norm.logpdf(Xval, loc=means0, scale=stds0), axis=1)
ll_1 = np.log(prob_c1) + np.sum(norm.logpdf(Xval, loc=means1, scale=stds1), axis=1)

# Your predicted class is 0 if class 0 has the highest log-likelihood, and 1 if class 1 
# has the highest log-likelihood
classes = ll_0 < ll_1

## Part (b) Checking results
We now compare our results with the sklearn implementation. 

In [ ]:
##Import the classifier GaussianNB from sklearn.naive_bayes
from sklearn.naive_bayes import GaussianNB
# Instantiate the classifier (use the parameter var_smoothing=0.0),
# fit, and predict the classes
gnb = GaussianNB(var_smoothing=0.)
gnb.fit(Xtrain, Ytrain)

val_preds = gnb.predict(Xval)

In [ ]:
# Compare your predicted classes with those of the sklearn implementation.
# If they are not identical, this may be due to some differences in parameter setting. 
# They should be almost all the same, however.

np.all(val_preds == classes)

## Part (c) Comparing k-nearest neighbours and Gaussian naive Bayes
Now retrain the naive Bayes classifier using the original training set `Xtr`, `Ytr`.
Also, retrain the k-nearest neighbours classifier using `Xtr` and `Ytr`. Use the value of $k$ that you decided on using cross-validation.
You can use the sklearn implementations of knn and naive Bayes.

Compute the accuracy of the naive Bayes classifier over the training set and the held-out test set.

Compare with the accuracy of the k-nearest neighbours classifier on each set.

Is it clear which classifier is the best on this dataset? why or why not?

In [ ]:
##TODO## Write your answer here
knn = KNeighborsClassifier(n_neighbors=8) #8 was the best performing value on the validation set

gnb.fit(Xtr, Ytr)
knn.fit(Xtr, Ytr)

gnb_tr = gnb.predict(Xtr)
gnb_test = gnb.predict(Xtest)

knn_tr = knn.predict(Xtr)
knn_test = knn.predict(Xtest)

print(f'\tTrain\t Test')
print(f'GNB\t{accuracy_score(Ytr, gnb_tr):.4f}\t{accuracy_score(Ytest, gnb_test):.4f}')
print(f'kNN\t{accuracy_score(Ytr, knn_tr):.4f}\t{accuracy_score(Ytest, knn_test):.4f}')


## Part (d) Using cross-validation for statistical validation
Earlier we used cross-validation to select the model parameters we would be using. We can also use it another way: to provide statistical information about which model is best. We will set up cross-validation on the whole dataset, with 10 folds.

 - Compute the accuracy for each model on the test set on each fold.
 - Calculate the mean accuracy across folds. Which model performs best?
 - Make a box-plot of the spread of scores of each model. Is there a clear difference between model performance?
 - Perform a paired t-test on the accuracy scores. What can you conclude about the performance of the two models?


In [ ]:
##TODO##
# Set up a k-fold cross-validation with 10 folds
cv = KFold(n_splits=10, shuffle=True)

In [ ]:
##TODO##
scoresknn=[]
scoresnb=[]

# For each fold, fit each model on the training data 
# and compute accuracy on the test data.
for train_index, test_index in cv.split(X):
    Xtr, Xtest = X[train_index], X[test_index]
    Ytr, Ytest = Y[train_index], Y[test_index]
    modelknn=knn.fit(Xtr, Ytr)
    modelnb=gnb.fit(Xtr, Ytr)
    Ypredknn=modelknn.predict(Xtest)
    Yprednb=modelnb.predict(Xtest)
    scoresknn.append(accuracy_score(Ytest,Ypredknn))
    scoresnb.append(accuracy_score(Ytest,Yprednb))

In [ ]:
# Compute the mean and standard devation of the accuracies for each model.
# Does one model perform better?
print(f'knn: mean={np.mean(scoresknn)}, sd={np.std(scoresknn)}')
print(f'nb: mean={np.mean(scoresnb)},sd={np.std(scoresnb)}')

In [ ]:
# Make a boxplot of the accuracy scores. Is there a clear difference between the models?
box_plot_data=[scoresknn,scoresnb]
plt.boxplot(box_plot_data,labels=['knn','nb'])
plt.show()

The box plots show that there is no clear difference in performance between the two models

In [ ]:
# Perform a paired t-test (you can use the function scipy.stats.ttest_rel). 
# What do you conclude about the performance of the two models?
from scipy.stats import ttest_rel
print(ttest_rel(scoresknn,scoresnb))

The outcome of the paired t-test shows that the performance of the models are not significantly different

## Linear Regression
In linear regression we make the assumption that the data $(x_i, y_i)$ can be modelled by a function of the form
$$ \hat{y_i} = f(\vec{x}_i)= \sum_j a_j x_{ij}  + b_i$$

Recall that we can express this in a matrix format by:
$$ \hat{\vec{y}} = f(X)= X\Theta$$

where 
$$ X=\begin{pmatrix}
x_{1,1} & x_{1,2} & \ldots & x_{1,n} &1 \\
\vdots & \vdots & \ldots & \vdots & \vdots \\
x_{N,1} & x_{N,2} & \ldots & x_{N,n} & 1
\end{pmatrix}, \quad \vec{y}=\begin{pmatrix} y_1 \\ \vdots \\y_N \end{pmatrix}, \quad \Theta=\begin{pmatrix} a_1 \\ \vdots \\a_n\\b \end{pmatrix}$$

We saw in lectures that the optimal value of $\Theta$ is given by setting
$$ \Theta = (X^T X)^{-1} X^T \vec{y}$$

The quantity $(X^T X)^{-1} X^T$ is called the psuedoinverse of X, and can be computed using the function `np.linalg.pinv`.

We will (a) perform a linear regression on the diabetes dataset. You can load this dataset using the function `load_diabetes` from `sklearn.datasets`. (b) compute the mean squared error and the R^2, and (c) compare your results with the built in function in sklearn (`sklearn.linear_model.LinearRegresion()`). You should get the same results.

In [ ]:
# import statments here
import math
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## Part (a) Implementing linear regression

In [ ]:
# Load the diabetes dataset
diabetes = datasets.load_diabetes()
X = diabetes.data
Y = diabetes.target

# Split the dataset into training, validation and test, using test_size=0.2
Xtr, Xtest, Ytr, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtr, Ytr, test_size=0.2, random_state=0)

In [ ]:
# Add a column of ones to your Xtrain and Xtest for the intercept term
Wtrain=np.hstack([Xtrain, np.ones((len(Xtrain),1))])
Wval=np.hstack([Xval, np.ones((len(Xval),1))])
Wtest=np.hstack([Xtest, np.ones((len(Xtest),1))])

In [ ]:
# Calculate the value of the coefficients theta. You can use the function np.linalg.pinv
theta = np.linalg.pinv(Wtrain).dot(Ytrain)

## Part (b) Computing performance metrics

In [ ]:
# Make a prediction on the test set by applying the coefficients theta to the test set
my_pred = Wval.dot(theta)

In [ ]:
# Calculate the mean squared error and the R^2. 
# You can use the built in functions from sklearn
print(f'mean squared error= {mean_squared_error(my_pred, Yval):.4f}')
print(f'R^2={r2_score(my_pred, Yval):.4f}')

## Part (c) Checking results
Compare your results with the built in function `sklearn.linear_model.LinearRegression()`

In [ ]:
regr = linear_model.LinearRegression()

In [ ]:
regr.fit(Xtrain, Ytrain)
pred = regr.predict(Xval)
print(f'mean squared error= {mean_squared_error(pred, Yval):.4f}')
print(f'R^2={r2_score(pred, Yval):.4f}')

Visualise the perfomance of the regression by plotting your predicted values vs target values on a scatter plot, and drawing a line y=x. If all predictions were perfect, the predicted values would lie on the line.

In [ ]:

plt.ylim(20, 350)
plt.xlim(20, 350)
plt.scatter(pred,Yval,color='black')
x = np.linspace(20,350,100) 
y=x
plt.plot(x, y,color='blue')

# Extra question: Polynomial regression
The term 'linear' in linear regression refers only to the coefficients $\theta$. We can in fact compute polynomial terms in the data and perform linear regression over this extended dataset to get a better fit to the data.

To compute polynomial terms in the data automatically, you can use the class `sklearn.preprocessing.PolynomialFeatures`. To find out how to use it, look at the guidance (you can type `help(PolynomialFeatures)` once you have imported it).

The following small dataset (in the cell below) gives a relationship between temperature and yield for an experiment. Use cross-validation to select the degree of the polynomial that best fits this data.

Plot the mean squared error against degree on the training set and on the validation set. Which degree of polynomial best fits this data?

In [ ]:
X = np.array([50,50,50,70,70,70,80,80,80,90,90,90,100,100,100]).reshape(-1, 1)
y = np.array([3.3,2.8,2.9,2.3,2.6,2.1,2.5,2.9,2.4,3,3.1,2.8,3.3,3.5,3]).reshape(-1, 1)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
Xtr, Xtest, Ytr, Ytest = train_test_split(X, y, test_size=0.2, random_state=0)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtr, Ytr, test_size=0.2, random_state=0)
mse_tr = []
mse_val = []
max_deg = 10
for i in range(max_deg):
    poly=PolynomialFeatures(degree=i+1)
    Xtrain_new = poly.fit_transform(Xtrain) 
    Xval_new = poly.fit_transform(Xval)
    regr.fit(Xtrain_new, Ytrain)
    pred_tr = regr.predict(Xtrain_new)
    pred_v = regr.predict(Xval_new)
    mse_tr.append(mean_squared_error(pred_tr, Ytrain))
    mse_val.append(mean_squared_error(pred_v, Yval))
    

In [ ]:
plt.plot(range(1, max_deg+1), mse_tr, label='Training')
plt.plot(range(1, max_deg+1), mse_val, label='Validation')
plt.legend()
plt.xlabel('Degree')
plt.ylabel('MSE')

In [ ]:
Xtest_new = poly.fit_transform(Xtest)
pred_test = regr.predict(Xtest_new)
mean_squared_error(pred_test, Ytest)